This notebook performs profile aggregation.

In [1]:
import argparse
import pathlib

import pandas as pd
from pycytominer import aggregate

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

In [2]:
if not in_notebook:
    argparser = argparse.ArgumentParser()
    argparser.add_argument(
        "--patient",
        type=str,
        required=True,
        help="Patient ID to process, e.g. 'P01'",
    )
    args = argparser.parse_args()
    patient = args.patient

else:
    patient = "NF0014"

In [3]:
# pathing
sc_fs_path = pathlib.Path(
    f"../../data/{patient}/image_based_profiles/3.sc_fs_profiles.parquet"
).resolve(strict=True)
organoid_fs_path = pathlib.Path(
    f"../../data/{patient}/image_based_profiles/3.organoid_fs_profiles.parquet"
).resolve(strict=True)


# output path
sc_agg_well_output_path = pathlib.Path(
    f"../../data/{patient}/image_based_profiles/4.sc_agg_well_level_profiles.parquet"
).resolve()
sc_agg_well_parent_organoid_output_path = pathlib.Path(
    f"../../data/{patient}/image_based_profiles/4.sc_agg_well_parent_organoid_level_profiles.parquet"
).resolve()
sc_consensus_output_path = pathlib.Path(
    f"../../data/{patient}/image_based_profiles/4.sc_consensus_profiles.parquet"
).resolve()

organoid_agg_well_output_path = pathlib.Path(
    f"../../data/{patient}/image_based_profiles/4.organoid_agg_well_level_profiles.parquet"
).resolve()
organoid_consensus_output_path = pathlib.Path(
    f"../../data/{patient}/image_based_profiles/4.organoid_consensus_profiles.parquet"
).resolve()

In [4]:
# read in the data
sc_fs = pd.read_parquet(sc_fs_path)
organoid_fs = pd.read_parquet(organoid_fs_path)

### Aggregate the single cell profiles
We will aggregated with a few different stratifications:
1. Well
2. Well and parent organoid
3. Treatment - i.e. the consensus profile for each treatment

In [5]:
sc_fs.head()

,patient,object_id,unit,dose,treatment,image_set,Well,parent_organoid,Area.Size.Shape_Nuclei_EXTENT,Colocalization_Nuclei_AGP.BF_MIN.CORRELATION.COEFF,...,Texture_Cytoplasm_AGP_Information.Measure.of.Correlation.2_256.1,Texture_Cytoplasm_BF_Correlation_256.1,Texture_Cytoplasm_BF_Inverse.Difference.Moment_256.1,Texture_Cytoplasm_BF_Sum.Average_256.1,Texture_Cytoplasm_DNA_Contrast_256.1,Texture_Cytoplasm_DNA_Sum.Variance_256.1,Texture_Cytoplasm_ER_Information.Measure.of.Correlation.1_256.1,Texture_Cytoplasm_ER_Sum.Variance_256.1,Texture_Cytoplasm_Mito_Contrast_256.1,Texture_Cytoplasm_Mito_Sum.Variance_256.1
0,NF0014,70,uM,10,Mirdametinib,G8-1,G8,40,0.590410,-0.937207,...,0.390195,0.292325,0.028686,-0.034991,-0.743879,-0.644247,-0.132689,-0.616014,0.387111,0.210545
1,NF0014,122,uM,10,Mirdametinib,G8-1,G8,40,0.242355,0.366156,...,0.550148,0.502795,0.003052,-0.009418,-0.512470,-0.388735,-0.743222,-0.410225,0.233132,0.240877
2,NF0014,162,uM,10,Mirdametinib,G8-1,G8,40,0.427268,-0.703361,...,0.207164,0.237268,0.064626,-0.070724,0.097954,0.042600,-0.598672,-0.553485,-0.001264,-0.003908
3,NF0014,165,uM,10,Mirdametinib,G8-1,G8,40,-1.756083,-0.418309,...,-0.550337,-0.142304,0.154835,-0.154227,-0.878099,-0.726517,-0.326628,-0.580438,-0.426149,-0.489233
4,NF0014,182,uM,10,Mirdametinib,G8-1,G8,40,0.570591,1.021355,...,-0.124153,0.383889,0.111983,-0.101184,-0.646486,-0.469616,-1.041207,-0.500639,-0.322758,-0.206362


In [6]:
sc_metadata_columns = [
    "patient",
    "object_id",
    "unit",
    "dose",
    "treatment",
    "image_set",
    "Well",
    "parent_organoid",
]
sc_features_columns = [col for col in sc_fs.columns if col not in sc_metadata_columns]
sc_features_df = sc_fs.drop(columns=sc_metadata_columns, errors="ignore")

In [7]:
# stratification approach #1
sc_well_agg = aggregate(
    population_df=sc_fs,
    strata=["Well"],
    features=sc_features_columns,
    operation="median",
)
sc_well_agg.to_parquet(sc_agg_well_output_path, index=False)

# stratification approach #2
sc_well_parent_organoid_agg = aggregate(
    population_df=sc_fs,
    strata=["Well", "parent_organoid"],
    features=sc_features_columns,
    operation="median",
)
sc_well_parent_organoid_agg.to_parquet(
    sc_agg_well_parent_organoid_output_path, index=False
)
# stratification approach #3
sc_consensus = aggregate(  # a.k.a. consensus
    population_df=sc_fs,
    strata=["treatment"],
    features=sc_features_columns,
    operation="median",
)
sc_consensus.to_parquet(sc_consensus_output_path, index=False)

### Aggregate the organoid profiles
We will aggregated with a few different stratifications:
1. Well
2. Treatment - i.e. the consensus profile for each treatment

In [8]:
organoid_fs.head()

,patient,object_id,unit,dose,treatment,image_set,Well,single_cell_count,Area.Size.Shape_Organoid_VOLUME,Area.Size.Shape_Organoid_EXTENT,...,Texture_Organoid_BF_Variance_256.1,Texture_Organoid_DNA_Contrast_256.1,Texture_Organoid_DNA_Variance_256.1,Texture_Organoid_ER_Contrast_256.1,Texture_Organoid_ER_Information.Measure.of.Correlation.1_256.1,Texture_Organoid_ER_Sum.Average_256.1,Texture_Organoid_ER_Variance_256.1,Texture_Organoid_Mito_Contrast_256.1,Texture_Organoid_Mito_Sum.Average_256.1,Texture_Organoid_Mito_Variance_256.1
0,NF0014,40,uM,10,Mirdametinib,G8-1,G8,8,-1.718171,-1.664739,...,-1.334719,-1.068404,-1.112068,-0.635342,-2.238332,-1.169085,-0.882218,-0.972143,-1.376009,-0.987831
1,NF0014,24,uM,1,Fimepinostat,D5-1,D5,5,-2.085979,-2.484067,...,-0.427715,-0.455481,-1.543011,0.074815,5.133408,-1.179782,-0.995430,1.584246,-1.498039,-1.093515
2,NF0014,85,uM,1,Mirdametinib,F8-1,F8,9,0.130074,-0.663134,...,0.068257,-0.455344,-1.153488,0.648688,1.518773,0.786162,-0.364094,-0.886440,-0.923582,-1.193731
3,NF0014,18,nM,10,STAURO,G11-1,G11,1,-2.223655,0.265362,...,-1.677969,-1.018121,-1.357958,-0.786237,-2.838483,-1.520782,-1.059963,-0.601197,-1.196517,-0.730073
4,NF0014,24,uM,1,Binimetinib,G7-1,G7,6,-1.728123,-0.369563,...,-0.549648,-0.699480,-0.593020,-0.760946,-0.497278,-1.270120,-1.007393,-0.507220,-0.360881,0.078821


In [9]:
organoid_metadata_columns = [
    "patient",
    "object_id",
    "unit",
    "dose",
    "treatment",
    "image_set",
    "Well",
    "parent_organoid",
]
organoidfeatures_columns = [
    col for col in organoid_fs.columns if col not in organoid_metadata_columns
]
organoid_features_df = organoid_fs.drop(columns=sc_metadata_columns, errors="ignore")

In [10]:
# stratification approach #1
organoid_well_agg = aggregate(
    population_df=organoid_fs,
    strata=["Well"],
    features=organoidfeatures_columns,
    operation="median",
)
organoid_well_agg.to_parquet(organoid_agg_well_output_path, index=False)

# stratification approach #2
organoid_consensus = aggregate(  # a.k.a. consensus
    population_df=organoid_fs,
    strata=["treatment"],
    features=organoidfeatures_columns,
    operation="median",
)
organoid_consensus.to_parquet(organoid_consensus_output_path, index=False)